In [1]:
import geemap

In [2]:
import ee

# Initialize Earth Engine
ee.Initialize()

In [3]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
Map.add_basemap("Satellite")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# coordinates for Delhi, India
latitude_min = 28.5
latitude_max = 28.9
longitude_min = 77.0
longitude_max = 77.4

# Define the Area of Interest (AOI) using the coordinates
aoi = ee.Geometry.Rectangle([longitude_min, latitude_min, longitude_max, latitude_max])


In [6]:
Map.addLayer(aoi, {}, "Delhi")
Map.centerObject(aoi, 10);
Map

Map(center=[28.700019627096843, 77.19999999999956], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
# Access Landsat data for the defined AOI and date range
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterBounds(aoi).filterDate('2013-03-18', '2024-01-01').filter(ee.Filter.lt('CLOUD_COVER_LAND', 0.5))


# Print the number of images in the collection
print("Number of images:", collection.size().getInfo())

Number of images: 106


In [8]:

Map.addLayer(collection, {}, 'clear images');
Map


Map(center=[28.700019627096843, 77.19999999999956], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# D thermal bands (Bands 10 and 11) from Landsat image collection
Map.addLayer(collection, {'bands' : ['B10','B11']}, 'Thermal bands')
Map


Map(center=[28.700019627096843, 77.19999999999956], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
# Extract thermal bands (B10, B11)
thermal_bands = collection.select(['B10', 'B11'])

In [11]:
# Define function to convert thermal bands to temperature
def calculate_temperature(image):
    # Define constants for Landsat 8 thermal data
    K1_CONSTANT = ee.Number(774.8853)
    K2_CONSTANT = ee.Number(1321.0789)
    
    # Calculate thermal radiance
    radiance = image.multiply(0.0003342).add(0.1)
    
    # Calculate temperature in Celsius using GEE's logarithm function
    temperature = K2_CONSTANT.divide(radiance.subtract(1).multiply(0.0003342)).log().multiply(-1).multiply(0.00341802).add(273.15)
    
    # Convert numeric temperature values to an image
    return ee.Image.constant(temperature).rename('Temperature')

In [12]:
# Apply the conversion function to the collection
temperature_collection = thermal_bands.map(calculate_temperature)

# Print the processed collection
print(temperature_collection)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.rename",
              "arguments": {
                "input": {
                  "functionInvocationValue": {
                    "functionName": "Image.constant",
                    "arguments": {
                      "value": {
                        "functionInvocationValue": {
                          "functionName": "Number.add",
                          "arguments": {
                            "left": {
                              "functionInvocationValue": {
                                "functionName": "Number.multiply",
                                "arguments": {
                                  "left": {
                   

In [13]:
# Visualize the calculated temperatures on the map
temperature_vis = {
    'min': 273.15,  # Minimum temperature in Kelvin
    'max': 313.15,  # Maximum temperature in Kelvin
    'palette': ['blue', 'green', 'yellow', 'red']
}


In [14]:
Map.addLayer(temperature_collection.mean(), temperature_vis, 'Mean Temperature')
Map

EEException: reduce.mean: Error in map(ID=LC08_146040_20130513):
Number.divide, argument 'right': Invalid type.
Expected type: Number.
Actual type: Image<[B10, B11]>.